<a href="https://colab.research.google.com/github/peppopi/Piratas/blob/main/Execute_Order_66.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
!pip install imagehash

#@title ## 🏴‍☠️ ¡Conectar Drive y Carpeta, o crear Carpeta!

#@markdown Pon el nonbre de tu carpeta en donde queres guardar las imagenes: TomasAgilar
nombre_proyecto = "TomasAgilar" #@param {type:"string"}
project_name = nombre_proyecto.strip()

def create_folders():
    if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
        print("¡Ahoy, capitán! Por favor, elige un nombre de proyecto válido. Evitemos los espacios y los caracteres especiales en este viaje.")
    else:
        drive.mount('/content/drive')


        main_dir = "/content/drive/MyDrive/Dataset"  #@markdown Siguiendo el ejemplo el link de tu carpeta sera el siguiente: /content/drive/MyDrive/Dataset/TomasAgilar
        images_folder = os.path.join(main_dir, project_name)

        if os.path.exists(images_folder):


            print(f"¡Ahoy, marinero! Ya tienes la carpeta {images_folder} preparada para guardar tus imágenes. ¡Excelente trabajo!")
        else:
            os.makedirs(images_folder)
            print(f"⚓️ ¡Carpetas creadas! La carpeta para las imágenes está en {images_folder}. ¡Preparados para llenarla de tesoros!")

create_folders()
images_folder = os.path.join("/content/drive/MyDrive/Dataset", project_name)

In [2]:
#@markdown ### ⚔️ ¡Elimina todos los gifs de la base de datos! 🗡️ [Recuerda que este codigo elimina todo lo que no sea png,jpg y txt de la carpeta del nombre del prollecto ende : /content/drive/MyDrive/Dataset/TomasAgilar]
import os
images_folder = os.path.join("/content/drive/MyDrive/Dataset", project_name)
# Obtener la lista de archivos y carpetas en la ruta
file_list = os.listdir(images_folder)

# Recorrer todos los archivos y carpetas en la ruta
for file_name in file_list:
    file_path = os.path.join(images_folder, file_name)

    # Verificar si es un archivo
    if os.path.isfile(file_path):
        # Verificar si el archivo no tiene una de las extensiones permitidas
        if not file_name.lower().endswith((".txt", ".png", ".jpg")):
            # Eliminar el archivo
            os.remove(file_path)
            print(f"Archivo eliminado: {file_name}")

    # Verificar si es una carpeta
    elif os.path.isdir(file_path):
        # Eliminar la carpeta y su contenido de forma recursiva
        shutil.rmtree(file_path)
        print(f"Carpeta eliminada: {file_name}")

In [ ]:
#@markdown ### ⚔️ ¡El siguiente coddigo se usa para detectar imagenes similares  y eliminarlas! 🗡️


#@markdown Nuestra base de datos esta recontra contaminada con imagenes tan similares que no aportan nada al programa por lo que buscamos reucirlas, en general veras como tu set de datos pasa de 1000 imagenes a unas meras 50, y con  eso estamos bien.

#@markdown Cuanto mas alto sea el valor de similitud, menos probabilidades tendra de elimar imagenes no tan parecidas entre si, lo ideal es que limpie gran parte de la basura, lo recomendable ahora es hacer una copia de la carpeta con las imagenes de forma de poder verificar si las imagenes eliminadas fueron la corrrectas ( recorda que no buscamos ser muy especificos, no necistamos tanto)

#@markdown Cuando termines de eliminar las imagenes, rebiza el archivos de imagenes manualmente usando el siguiente programa: solo queremos usar su lector para ver si borramos lo corecto o debemos diminuir el similarity tresholder y borrar mas cosas. Tambien puedes borrar las que quedan manualmente usando la funcion de no recortar imagenes, pero recureda que la idea es hacerlo automaticamente: https://www.birme.net/?target_width=1000&target_height=900&no_resize=true&auto_focal=false


import os
import cv2
import imagehash
from PIL import Image
#@markdown Metodo de ImageHash - Investigacion relativa al estilo del eroge , inmediato
#@markdown
#@markdown 0 (No se eliminaran imagenes)
#@markdown
#@markdown 25: Elimina duplicadas de baja calidad - Valor recomendado para iniciar a eliminar
#@markdown
#@markdown 30: Comiensa a detectar los cambios de expreciones
#@markdown
#@markdown 40: Detectame correctamente imagenes similares con distintas expresiones
#@markdown
#@markdown 45: Comienzan a eliminarse las imagenes de cambio de ropa y poses identicas
#@markdown
#@markdown 55: Comienzan a eliminarse imagenes de alto valor
#@markdown
#@markdown 128 : Se eliminaran todas las imagenes

def calculate_ahash(image):
    # Cambiar el tamaño de la imagen a un tamaño fijo (8x8) para calcular el hash
    resized_image = cv2.resize(image, (8, 8))

    # Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Calcular el hash perceptual promedio (aHash)
    hash_value = imagehash.average_hash(Image.fromarray(gray_image))

    return hash_value

def normalize_difference(hash_diff):
    # Normalizar la diferencia de hashes a un rango de 0 a 128
    normalized_diff = 128 * (1 - hash_diff / 64)
    return normalized_diff

def remove_duplicates(folder_path, intensity_threshold):
    hashes = {}
    duplicates = []

    # Leer todas las imágenes en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            image_hash = calculate_ahash(image)

            # Comprobar si el hash ya existe
            is_duplicate = False
            for existing_hash in hashes:
                hash_diff = existing_hash - image_hash
                similarity = normalize_difference(hash_diff)
                if similarity <= intensity_threshold:
                    is_duplicate = True
                    duplicates.append(filename)
                    os.remove(image_path)
                    break

            if not is_duplicate:
                hashes[image_hash] = filename

    return duplicates

# Ruta de la carpeta que contiene las imágenes
NombreDelProllecto = "TomasAgilar" #@param {type:"string"}
folder_path = f"/content/drive/MyDrive/Dataset/{NombreDelProllecto}"
intensity_threshold = 35  #@param {type:"slider", min:0, max:128, step:5}

# Eliminar las imágenes duplicadas con la intensidad especificada
duplicates = remove_duplicates(folder_path, intensity_threshold)

print(f"Se eliminaron {len(duplicates)} imágenes duplicadas.")


def count_images(folder_path):
    count = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            count += 1
    return count

# Obtener la cantidad de imágenes que quedan en la carpeta
remaining_images = count_images(folder_path)

if remaining_images > 125:
    print("El programa tiene más de 125 imágenes, se recomienda aumentar un poco la intensidad.")
elif remaining_images < 40:
    print("El programa tiene menos de 40 imágenes, se recomienda la eliminación manual.")
else:
    print("El programa tiene una cantidad adecuada de imágenes.")

print(f"Quedan {remaining_images} imágenes en la carpeta.")

In [ ]:
import os

main_dir = images_folder
root_dir = main_dir
#@markdown ### ⚡️ Paso numero tres,  daremos los tags a las imagenes automaticamente con el siguiente programa
#@markdown Simpre usa este metodo, ya que los tags los usaremos para otra funcion ademas de entrenar el Lora. El codigo eliminara automaticamente todos los archivos GIF de la base de datos y cualquier cosa que no sea una imagen valida y un archivo de imagen valido en la carpeta especificada. Por lo que  usa una carpeta Limpia con solo las imaegnes a trabajar o los gif (y nunca uses un directorio con multiples carpetas)
metodo = "\u2694\uFE0F Anime"  #@param ["⚔️ Anime", "📷 Fotorealismo"]
method = metodo
#@markdown El umbral debe ser lo suficientemente bajo como para no cagarla y lo suficientemente alto para identificar lo que buscamos, en este caso diferenciar un fondo de un personaje.
umbral = 0.35  #@param {type:"slider", min:0.0, max:1.0, step:0.01}
tag_threshold = umbral
#@markdown No toques esto, no deberia afectar en nada
largo_minimo = 10  #@param {type:"number"}
caption_min = largo_minimo
largo_maximo = 100  #@param {type:"number"}
caption_max = largo_maximo


tags_folder = images_folder  # Ruta de la carpeta donde se guardan las imágenes

# Recorrer todos los archivos y carpetas en la ruta
for root, dirs, files in os.walk(tags_folder):
    for file_name in files:
        if file_name.endswith(".txt"):
            file_path = os.path.join(root, file_name)

            # Eliminar el archivo .txt
            os.remove(file_path)
            print(f"Archivo eliminado: {file_name}")

kohya = "/content/kohya-trainer"

if not os.path.exists(kohya):
    !git clone https://github.com/kohya-ss/sd-scripts {kohya}

if "⚔️ Anime" in method:
    if "step4a_installed_flag" not in globals():
        print("🏴‍☠️ Izando las velas y afilando los ganchos...\n")
        !pip -q install tensorflow==2.12.0 huggingface-hub==0.12.0 accelerate==0.15.0 transformers==4.26.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6 torchvision albumentations
        if not get_ipython().__dict__['user_ns']['_exit_code']:

            step4a_installed_flag = True
        else:
            print("❌ ¡A la bodega! ¡Error en la instalación! Pero seguiremos adelante, ¡arrr!")

    print("🚶‍♂️ ¡Sube a bordo, marinero! ¡Izamos anclas y comenzamos la travesía!\n")

    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    %env PYTHONPATH={kohya}
    !python {kohya}/finetune/tag_images_by_wd14_tagger.py \
        {images_folder} \
        --repo_id=SmilingWolf/wd-v1-4-swinv2-tagger-v2 \
        --model_dir={root_dir} \
        --thresh={tag_threshold} \
        --batch_size=8 \
        --caption_extension=.txt \
        --force_download

    if not get_ipython().__dict__['user_ns']['_exit_code']:
        print("🦜 ¡Excelente! Hemos encontrado verdaderos tesoros en tus imágenes. Pero antes de partir, ajustemos los detalles finales. Quitaré los guiones bajos de los tags y los prepararé para su uso.")

        from collections import Counter
        top_tags = Counter()
        for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
            with open(os.path.join(images_folder, txt), 'r') as f:
                tags = [t.strip() for t in f.read().split(",")]
                tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
            top_tags.update(tags)
            with open(os.path.join(images_folder, txt), 'w') as f:
                f.write(", ".join(tags))

        print("📊 ¡Tesoro encontrado! Aquí están las 50 etiquetas más comunes en tus imágenes:")
        print("\n".join(f"🏴‍☠️ {k} ({v})" for k, v in top_tags.most_common(50)))

else:  # 📷 Fotorealismo
    if "step4b_installed_flag" not in globals():
        print("🏴‍☠️ Izando las velas y afilando los ganchos...\n")
        !pip -q install timm==0.6.12 fairscale==0.4.13 transformers==4.26.0 requests==2.28.2 accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6
        if not get_ipython().__dict__['user_ns']['_exit_code']:

            step4b_installed_flag = True
        else:
            print("❌ ¡A la bodega! ¡Error en la instalación! Pero seguiremos adelante, ¡arrr!")

    print("🚶‍♂️ ¡Sube a bordo, marinero! ¡Izamos anclas y comenzamos la travesía!\n")

    os.chdir(kohya)
    %env PYTHONPATH={kohya}
    !python {kohya}/finetune/make_captions.py \
        {images_folder} \
        --beam_search \
        --max_data_loader_n_workers=2 \
        --batch_size=8 \
        --min_length={caption_min} \
        --max_length={caption_max} \
        --caption_extension=.txt

    if not get_ipython().__dict__['user_ns']['_exit_code']:
        import random
        captions = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
        sample = []
        for txt in random.sample(captions, min(10, len(captions))):
            with open(os.path.join(images_folder, txt), 'r') as f:
                sample.append(f.read())

        os.chdir(root_dir)
        %env PYTHONPATH=/env/python

        print("📊 ¡Tesoro encontrado! Aquí tienes algunos ejemplos de los subtítulos en tus imágenes:")
        print("\n".join([f"🏴‍☠️ {s}\n" for s in sample]))

print("\n🏴‍☠️ ¡La búsqueda ha concluido, marinero! Hemos terminado de describir tus imágenes, y ahora está en tus manos usar este valioso conocimiento para el bien de tu Lora. ¡A seguir navegando y descubriendo nuevos tesoros visuales!")

#Limpiador de Archivos molestos
import os
import shutil

# Obtener la lista de archivos y carpetas en la ruta
file_list = os.listdir(images_folder)

# Recorrer todos los archivos y carpetas en la ruta
for file_name in file_list:
    file_path = os.path.join(images_folder, file_name)

    # Verificar si es un archivo
    if os.path.isfile(file_path):
        # Verificar si el archivo no tiene una de las extensiones permitidas
        if not file_name.lower().endswith((".txt", ".png", ".jpg")):
            # Eliminar el archivo
            os.remove(file_path)
            print(f"Archivo eliminado: {file_name}")

    # Verificar si es una carpeta
    elif os.path.isdir(file_path):
        # Eliminar la carpeta y su contenido de forma recursiva
        shutil.rmtree(file_path)
        print(f"Carpeta eliminada: {file_name}")

In [ ]:
#@markdown ### ⚔️ ¡Elimina las imagenes con tags de baja calidad! 🗡️💣🏴‍☠️
#@markdown  El objetivo es poder reducir aun mas la base de datos sacano todos los background y quedarnos solo con los personajes, si la situacion lo permite es combeniente que eliminemos las imagenes con fondo negro irrelebantes y nos quedemos solo con las buenas, pero no todos los juegos dan esa opcion y no simpre combiene, depende del tipo de eroge
import os
#@markdown 🏴‍☠️ **Tag Maldito**: ¡Ingresa aquí el tag que debe ser erradicado! Este tag ha corrompido los tesoros y debe ser destruido. Por ejemplo, "no human", "balck background", "malvado"...
tag_maldito = "black background,simple background"  #@param {type:"string"}
# Tag que se desea buscar y eliminar
tags_folder= os.path.join("/content/drive/MyDrive/Dataset", project_name)
# Recorrer todos los archivos y carpetas en la ruta
# Convertir los tags malditos en una lista
tags_malditos = [tag.strip() for tag in tag_maldito.split(",")]

# Recorrer todos los archivos y carpetas en la ruta
for root, dirs, files in os.walk(tags_folder):
    for file_name in files:
        if file_name.endswith(".txt"):
            txt_file_path = os.path.join(root, file_name)

            # Verificar si el archivo .txt contiene algún tag maldito
            with open(txt_file_path, 'r') as f:
                tags = [s.strip() for s in f.read().split(",")]

            eliminar_imagen = False
            for tag in tags:
                if tag in tags_malditos:
                    eliminar_imagen = True
                    break

            if eliminar_imagen:
                # Eliminar la imagen correspondiente (jpg o png)
                base_name = os.path.splitext(file_name)[0]
                image_extensions = (".jpg", ".png")
                for extension in image_extensions:
                    image_path = os.path.join(root, base_name + extension)
                    if os.path.exists(image_path):
                        os.remove(image_path)
                        print(f"Imagen eliminada: {base_name + extension}")
                        break  # Solo se elimina una imagen si se encuentra

                # Eliminar el archivo .txt
                os.remove(txt_file_path)
                print(f"Archivo eliminado: {file_name}")

In [ ]:
#@markdown ### ⚔️ ¡Luego de ejecutar todos estos codigos deberias tener una base datos para entrenar tus Loras de pocas iamgenes y altisima calidad, de forma tal de que logres sacar todos los personajes y estilo del artista! 🗡️💣🏴‍☠️